# **redis-feast-gcp**: 00 - Feature Store Intro

In this notebook, we will explore how a Feature Store works using Feast.

**This notebook assumes that you've already set up your Feature Store in GCP using the `setup` instructions in the README.**

![architecture](img/redis-feast-gcp-architecture.png)

## Fetching Historical Data

The Feast SDK allows us to pull historical data to generate a training dataset. In this cae, the data comes from BigQuery tables, but unbeknownst to the user.

To make things simpler, we use the [`DataFetcher`](utils/data_fetcher.py) class that wraps the Feast Python client.

In [1]:
from feature_store.repo import config
from feature_store.utils import (
    DataFetcher,
    storage
)

# Load Feast feature store
fs = storage.get_feature_store(
    config_path=config.REPO_CONFIG,
    bucket_name=config.BUCKET_NAME
)

# Load DataFetcher utility class
data_fetcher = DataFetcher(fs)

Feast is an open source project that collects anonymized error reporting and usage statistics. To opt out or learn more see https://docs.feast.dev/reference/usage


FeatureServiceNotFoundException: Feature service serving_features does not exist in project feature_store

In [ ]:
# The data fetcher class contains the list of input features to use for model training
print(data_fetcher.X_cols)

print(data_fetcher.y_col)

In [ ]:
# Fetch historical training data with Feast
ds = data_fetcher.get_training_data(
    entity_query=f"""
        select
            state,
            date as event_timestamp
        from
            {config.BIGQUERY_DATASET_NAME}.{config.WEEKLY_VACCINATIONS_TABLE}
    """
)

# Clean up any nulls
ds.dropna(inplace=True)
ds.sort_values(['event_timestamp', 'state'], axis=0, inplace=True)

In [ ]:
# Subset the data to the state of Virginia (in the US)
virginia = ds[ds.state == 'Virginia']

In [ ]:
# Inspect part of the dataset for Virginia
virginia.head()

In [ ]:
virginia.tail()

In [ ]:
virginia.weekly_vaccinations_count.describe()

In [ ]:
# Investigate trends across features for prediction weekly vaccinations count!
virginia.plot.scatter(y='weekly_vaccinations_count', x='lag_2_vaccine_interest')

## Low latency feature retrieval

To make a prediction in real-time, we need to do the following:

1. Fetch our features from Redis using the Feast client
2. Pass these features to our model
3. Return these predictions to the user

Here is an example pulling the latest features data from Redis (online store) for the state of *Virginia*.

In [ ]:
# Fetch online feature
online_feature = data_fetcher.get_online_data(state="Virginia")

In [ ]:
online_feature

### Alternatives for Feature Retrieval
- Instead of using the Feast SDK (and our Data Fetcher class), you can also setup and use the [Python feature server](https://docs.feast.dev/reference/feature-servers/python-feature-server).
- If you need the best in class performance for feature retrieval and don't mind the extra setup, Feast also supports a [Go feature server](https://docs.feast.dev/reference/feature-servers/go-feature-server) that is much faster than standard Python.

If included in the project, you can run these HTTP (or gRPC) servers with `feast serve` command. The doc links above show how to set that up.